# Plots
> Common plots used in the dashboards

In [ ]:
#| default_exp plots

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| exporti
import json, os, math
import itertools as it
from collections import defaultdict

import numpy as np
import pandas as pd
import datetime as dt

from typing import List, Tuple, Dict, Union, Optional

import altair as alt
import scipy as sp
import scipy.stats as sps
from scipy.cluster import hierarchy
from KDEpy import FFTKDE
import arviz as az

from salk_toolkit.utils import *
from salk_toolkit.io import extract_column_meta, read_json

from matplotlib import font_manager
from PIL import ImageFont

# Machinery

In [ ]:
#| export
registry = {}
registry_meta = {}

In [ ]:
# For later testing to work with .pp functions in the notebook, 
# we need to make sure we use the registry defined in the .py file
from salk_toolkit.plots import registry, registry_meta

In [ ]:
#| export

stk_plot_defaults = { 'data_format': 'longform' }

# Decorator for registering a plot type with metadata
def stk_plot(plot_name, **r_kwargs):
    
    def decorator(gfunc):
        # In theory, we could do transformations in wrapper
        # In practice, it would only obfuscate already complicated code
        #def wrapper(*args,**kwargs) :
        #    return gfunc(*args,**kwargs)

        # Register the function
        registry[plot_name] = gfunc
        registry_meta[plot_name] = { **stk_plot_defaults, **r_kwargs }
        
        return gfunc
    
    return decorator

def stk_deregister(plot_name):
    del registry[plot_name]
    del registry_meta[plot_name]

def get_plot_fn(plot_name):
    return registry[plot_name]

def get_plot_meta(plot_name):
    return registry_meta[plot_name].copy()

In [ ]:
#| export

# First is weight if not matching, second if match
# This is very much a placeholder right now
priority_weights = {
    'likert': [-10000, 100],
    'continuous': [0, 100],
    'draws': [0,50],
    'question': [0, 100],
    'ordered': [-10000,100],
    'ordered_factor':[-10000,100],
    'requires_factor':[-10000,0],
    'factor_meta':[-10000,0]
}

def calculate_priority(plot_meta, match):
    base = 0
    if 'factor_meta' in plot_meta: # Somewhat hacky way of adding this but it works
        if len(set(plot_meta['factor_meta']) - set(match['factor_meta']))>0: base += priority_weights['factor_meta'][0]
                                                                 
    return base + sum([ priority_weights[k][vod(match,k) or 0] for k, v in plot_meta.items() if k not in ['factor_meta'] and k in priority_weights and v ])

def calculate_impossibilities(plot_meta, match):
    return [ k for k, v in plot_meta.items() if k not in ['factor_meta'] and k in priority_weights and v and priority_weights[k][vod(match,k) or 0]<0 ]

# Get a list of plot types matching required spec
def matching_plots(args, df, data_meta, details=False, list_hidden=False):
    
    rc = args['res_col']
    col_meta = extract_column_meta(data_meta)
    
    match = {
        'draws': ('draw' in df.columns),
        'likert': vod(col_meta[rc],'likert'),
        'question': (rc not in df.columns),
        'continuous': ('categories' not in col_meta[rc]),
        'ordered': vod(col_meta[rc],'ordered'),
        'ordered_factor': (vod(args,'factor_cols',[])==[]) or not vod(args,'internal_facet') or vod(col_meta[args['factor_cols'][0]],'ordered'),
        'requires_factor': (vod(args,'factor_cols',[])!=[]) and vod(args,'internal_facet'),
        'hidden': list_hidden
    }
    
    if vod(args,'convert_res')=='continuous' and vod(col_meta[rc],'ordered'):
        match = {**match,'continuous':True,'ordered':False,'likert':False}
    
    match['factor_meta'] = col_meta[args['factor_cols'][0]] if match['requires_factor'] else []
    
    res = [ ( pn, calculate_priority(get_plot_meta(pn),match), calculate_impossibilities(get_plot_meta(pn),match)) for pn in registry.keys() ]
    
    if details: return { n: (p, i) for (n, p, i) in res } # Return dict with priorities and failure reasons
    else: return [ n for (n,p,i) in sorted(res,key=lambda t: t[1], reverse=True) if p >= 0 ] # Return list of possibilities in decreasing order of fit

In [ ]:
# For tests, set up a data file and import from pp
from salk_toolkit.pp import e2e_plot
data_file = '../samples/w25_bootstrap.parquet'

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
#| export

# Create and register a continuous version of the cat_fn_name plot with 'question' dimension replacing category
# This assumes the plot can display not just percentages but all real numbers
def register_stk_cont_version(cat_fn_name):
    cat_fn, cat_fn_meta = get_plot_fn(cat_fn_name), get_plot_meta(cat_fn_name)
    @stk_plot(f'{cat_fn_name}-cont', **{**cat_fn_meta, **{'continuous':True, 'question':True} })
    def cont(data, value_col='value', question_col='question', question_color_scale=alt.Undefined, question_order=alt.Undefined, **kwargs):
        
        # Remap certain args while keeping everything else intact
        kwargs = {**kwargs, **{'data':data, 'cat_col':question_col, 'cat_order': question_order, 'value_col':value_col, 'color_scale':question_color_scale}}
        
        return cat_fn(**clean_kwargs(cat_fn,kwargs))
    return cont

## Small utility function to wrap horizontal legends properly

In [ ]:
#| exporti

# Find a sensible approximation to the font used in vega/altair
font = font_manager.FontProperties(family='sans-serif', weight='regular')
font_file = font_manager.findfont(font)
legend_font = ImageFont.truetype(font_file,10)

In [ ]:
#| export

# Legends are not wrapped, nor is there a good way of doing accurately it in vega/altair
# This attempts to estimate a reasonable value for columns which induces wrapping
def estimate_legend_columns_horiz_naive(cats, width):
    max_str_len = max(map(len,cats))
    n_cols = max(1,width//(15+5*max_str_len))
    # distribute them roughly equally to avoid last row being awkwardly shorter
    n_rows = int(math.ceil(len(cats)/n_cols))
    return int(math.ceil(len(cats)/n_rows))

# More sophisticated version that looks at lengths of individual strings across multiple rows
# ToDo: it should max over each column separately not just look at max(sum(row)). This is close enough though.
def estimate_legend_columns_horiz(cats, width):
    max_cols, restart = len(cats), True
    lens = list(map(lambda s: 15+legend_font.getlength(s),cats))
    while restart:
        restart, rl, cc = False, 0, 0
        for l in lens:
            if cc >= max_cols: # Start a new row
                rl, cc = l, 1
            elif rl + l > width: # Exceed width - restart
                max_cols = cc
                # Start from beginning every thime columns number changes
                # This is because what ends up in second+ rows depends on length of first
                restart = True
            else: # Just append to existing row
                rl += l
                cc += 1
                
    # For very long labels just accept we can't do anything
    max_cols = max(max_cols,1)

    # distribute them roughly equally to avoid last row being awkwardly shorter
    n_rows = int(math.ceil(len(cats)/max_cols))
    return int(math.ceil(len(cats)/n_rows))

In [ ]:
assert estimate_legend_columns_horiz(["Keskerakond", "EKRE", "Reformierakond", "Isamaa", "SDE", "Rohelised", "Eesti 200", "Parempoolsed", "Other", "None of the parties", "No opinion"],800) == 11

# Plots

## Data options:

For 'data' keyword:
 - raw: 
 - longform: [(draw),(factor1),(factor2), category, value] (default)
 - table: [(draw),(factor1),(factor2), c1, c2, c3, ...]
In case of question groups, question is factor 1

Other keywords:
 - draws: if it requires draws to be present (or simulated)
 - likert: requires category to be symmetric with the neutral/na value in middle for odd number of levels
 - question: if it expects a "question" column, i.e. to be over a group of questions
 - ordered: requires category to be ordered
 - ordered_factor: requires internal factor to be ordered (usually meant for it to be time)

In [ ]:
#| export
@stk_plot('boxplots', data_format='longform', draws=True)
def boxplots(data, cat_col, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, factor_color_scale=alt.Undefined, factor_order=alt.Undefined, val_format='%', width=800, tooltip=[]):
    if val_format[-1] == '%': # Boxplots being a compound plot, this workaround is needed for axis & tooltips to be proper
        data[value_col]*=100
        val_format = val_format[:-1]+'f'
    
    shared = {
        'y': alt.Y(f'{cat_col}:N', title=None, sort=cat_order),

        **({
            'color': alt.Color(f'{cat_col}:N', scale=color_scale, legend=None)    
            } if not factor_col else {
                'yOffset':alt.YOffset(f'{factor_col}:N', title=None, sort=factor_order), 
                'color': alt.Color(f'{factor_col}:N', scale=factor_color_scale, 
                                   legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(factor_order,width)))
            })
    }
    
    base = alt.Chart(round(data, 2))
    
    # This plot is here because boxplot does not draw if variance is very low, so this is the backup
    tick_plot = base.mark_tick(thickness=3).encode(
        x=alt.X(f'mean({value_col}):Q'),
        tooltip=[alt.Tooltip(f'mean({value_col}):Q')] + tooltip[1:],
        **shared
    )
    
    box_plot = base.mark_boxplot(
        clip=True,
        #extent='min-max',
        outliers=False
    ).encode(
        x=alt.X(
            f'{value_col}:Q',
            title=value_col,
            axis=alt.Axis(format=val_format)
            ),
        tooltip=tooltip[1:],
        **shared,
    )
    return tick_plot + box_plot

register_stk_cont_version('boxplots')

In [ ]:
e2e_plot({
    'res_col' : 'party_preference',
    'factor_cols': ['age_group'],
    'filter': {},
    'plot': 'boxplots',
    'internal_facet': True
},data_file)

In [ ]:
#| export
@stk_plot('columns', data_format='longform', draws=False)
def columns(data, cat_col, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, factor_color_scale=alt.Undefined, factor_order=alt.Undefined, val_format='%', width=800, tooltip=[]):
    plot = alt.Chart(round(data, 3), width = 'container' \
    ).mark_bar().encode(
        x=alt.X(
            f'{value_col}:Q',
            title=value_col,
            axis=alt.Axis(format=val_format),
            #scale=alt.Scale(domain=[0,30]) #see lõikab mõnedes jaotustes parema ääre ära
        ),
        y=alt.Y(f'{cat_col}:N', title=None, sort=cat_order),
        tooltip = tooltip,
        **({
                'color': alt.Color(f'{cat_col}:N', scale=color_scale, legend=None)    
            } if not factor_col else {
                'yOffset':alt.YOffset(f'{factor_col}:N', title=None, sort=factor_order),
                'color': alt.Color(f'{factor_col}:N', scale=factor_color_scale,
                                    legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(factor_order,width)))
            }),
    )
    return plot

register_stk_cont_version('columns')

In [ ]:
pp = e2e_plot({
    'res_col' : 'thermometer',
    'factor_cols': ['nationality'],
    'filter': {},
    'plot': 'columns-cont',
    'internal_facet': True
},data_file,dry_run=True)
pp['data'].columns

In [ ]:
e2e_plot({
    'res_col' : 'thermometer',
    'factor_cols': ['nationality'],
    'filter': {},
    'plot': 'columns-cont',
    'internal_facet': True
},data_file)

In [ ]:
#| export
@stk_plot('stacked_columns', data_format='longform', draws=False, requires_factor=True, agg_fn='sum', args={'normalized':'bool'})
def stacked_columns(data, cat_col, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, factor_color_scale=alt.Undefined, factor_order=alt.Undefined, n_datapoints=1, val_format='%', width=800, normalized=False, tooltip=[]):
    
    odims = [ c for c in data.columns if c not in [value_col,cat_col,factor_col] ]
    data[value_col] = data[value_col]/n_datapoints
    
    ldict = dict(zip(factor_order, range(len(factor_order))))
    data['f_order'] = data[factor_col].astype('object').replace(ldict).astype('int')
    
    plot = alt.Chart(round(data, 3), width = 'container' \
    ).mark_bar().encode(
        x=alt.X(
            f'{value_col}:Q',
            title=value_col,
            axis=alt.Axis(format=val_format),
            **({'stack':'normalize'} if normalized else {})
            #scale=alt.Scale(domain=[0,30]) #see lõikab mõnedes jaotustes parema ääre ära
        ),
        y=alt.Y(f'{cat_col}:N', title=None, sort=cat_order),
        tooltip = tooltip,
        **({
                'color': alt.Color(f'{cat_col}:N', scale=color_scale, legend=None)    
            } if not factor_col else {
                'order': alt.Order('f_order:O'),
                'color': alt.Color(f'{factor_col}:N', scale=factor_color_scale,
                                    legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(factor_order,width)))
            }),
    )
    return plot

register_stk_cont_version('stacked_columns')

In [ ]:
e2e_plot({
  "res_col": "party_preference",
  "factor_cols": [
    "EKRE"
  ],
  "internal_facet": True,
  "plot": "stacked_columns",
  "plot_args": {
    "normalized": False
  },
  "filter": {}
},data_file)

In [ ]:
#| export
@stk_plot('diff_columns', data_format='longform', draws=False, requires_factor=True, args={'sort_descending':'bool'})
def diff_columns(data, cat_col, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, factor_color_scale=alt.Undefined, val_format='%', sort_descending=False, tooltip=[]):
    
    ind_cols = list(set(data.columns)-{value_col,factor_col})
    factors = data[factor_col].unique() # use unique instead of categories to allow filters to select the two that remain
    
    idf = data.set_index(ind_cols)
    diff = (idf[idf[factor_col]==factors[1]][value_col]-idf[idf[factor_col]==factors[0]][value_col]).reset_index()
    
    if sort_descending: cat_order = list(diff.sort_values(value_col,ascending=False)[cat_col])
    
    plot = alt.Chart(round(diff, 3), width = 'container' \
    ).mark_bar().encode(
        y=alt.Y(f'{cat_col}:N', title=None, sort=cat_order),
        x=alt.X(
            f'{value_col}:Q',
            title=f"{factors[1]} - {factors[0]}",
            axis=alt.Axis(format=val_format, title=f"{factors[0]} <> {factors[1]}"),
            #scale=alt.Scale(domain=[0,30]) #see lõikab mõnedes jaotustes parema ääre ära
            ),
        
        tooltip=[
            alt.Tooltip(f'{cat_col}:N'),
            alt.Tooltip(f'{value_col}:Q',format=val_format, title=f'{value_col} difference')
            ],
        color=alt.Color(f'{cat_col}:N', scale=color_scale, legend=None)    
    )
    return plot

register_stk_cont_version('diff_columns')

In [ ]:
e2e_plot({
    'res_col' : 'thermometer',
    'factor_cols': ['age_group'],
    'filter': { 'age_group': ('25-34','35-44')},
    'plot': 'diff_columns-cont',
    'internal_facet': True,
    'plot_args': { 'sort_descending': True }
},data_file)

In [ ]:
#| export

# The idea was to also visualize the size of each cluster. Currently not very useful, may need to be rethought

@stk_plot('massplot', data_format='longform', draws=False, group_sizes=True, hidden=True)
def massplot(data, cat_col, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, factor_color_scale=alt.Undefined, factor_order=alt.Undefined, n_datapoints=1, val_format='%', width=800, tooltip=[]):

    data['group_size']=(data['group_size']/n_datapoints)#.round(2)

    plot = alt.Chart(round(data, 3), width = 'container' \
    ).mark_circle().encode(
        y=alt.Y(f'{cat_col}:N', title=None, sort=cat_order),
        x=alt.X(
            f'{value_col}:Q',
            title=value_col,
            axis=alt.Axis(format=val_format),
            #scale=alt.Scale(domain=[0,30]) #see lõikab mõnedes jaotustes parema ääre ära
            ),
        size=alt.Size('group_size:Q', legend=None, scale=alt.Scale(range=[100, 500])),
        opacity=alt.value(1.0),
        stroke=alt.value('#777'),
        tooltip = tooltip + [ alt.Tooltip('group_size:N',format='.1%',title='Group size') ],
        #tooltip=[
        #    'response:N',
            #alt.Tooltip('mean(support):Q',format='.1%')
        #    ],
        **({
                'color': alt.Color(f'{cat_col}:N', scale=color_scale, legend=None)    
            } if not factor_col else {
                'yOffset':alt.YOffset(f'{factor_col}:N', title=None, sort=factor_order), 
                'color': alt.Color(f'{factor_col}:N', scale=factor_color_scale,
                                legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(factor_order,width)))
            }),
    )
    return plot

register_stk_cont_version('massplot')

In [ ]:
e2e_plot({
    'res_col' : 'trust',
    'factor_cols': ['party_preference'],
    'filter': {},
    'plot': 'massplot-cont',
    'internal_facet': True,
    'convert_res': 'continuous'
},data_file)#,dry_run=True)['data']

In [ ]:
#| export

# Make the likert bar pieces
def make_start_end(x,value_col,cat_col,cat_order):
    #print("######################")
    #print(x)
    if len(x) != len(cat_order):
        # Fill in missing rows with value zero so they would just be skipped
        x = pd.merge(pd.DataFrame({cat_col:cat_order}),x,on=cat_col,how='left').fillna({value_col:0})
    mid = len(x)//2
        
    if len(x)%2==1: # odd:
        scale_start=1.0
        x_mid = x.iloc[[mid],:]
        x_mid.loc[:,['start','end']] = -scale_start
        x_mid.loc[:,'end'] = -scale_start+x_mid[value_col]
        nonmid = [ i for i in range(len(x)) if i!=mid ]
        x_mid = [x_mid]
    else: # even - no separate mid
        nonmid = np.arange(len(x))
        x_mid = []
    
    x_other = x.iloc[nonmid,:].copy()
    x_other.loc[:,'end'] = x_other[value_col].cumsum() - x_other[:mid][value_col].sum()
    x_other.loc[:,'start'] = (x_other[value_col][::-1].cumsum()[::-1] - x_other[mid:][value_col].sum())*-1
    res = pd.concat([x_other] + x_mid).dropna() # drop any na rows added in the filling in step
    #print(res)
    return res

@stk_plot('likert_bars',data_format='longform',question=True,draws=False,likert=True)
def likert_bars(data, cat_col, cat_order=alt.Undefined, value_col='value', question_col='question', question_order=alt.Undefined, color_scale=alt.Undefined, factor_col=None, factor_color_scale=alt.Undefined, factor_order=alt.Undefined, tooltip=[], outer_factors=[]):
    gb_cols = [ c for c in outer_factors+[question_col,factor_col] if c is not None ] # There can be other extra cols (like labels) that should be ignored
    options_cols = list(data[cat_col].dtype.categories) # Get likert scale names
    bar_data = data.groupby(gb_cols, group_keys=False, observed=False)[data.columns].apply(make_start_end, value_col=value_col,cat_col=cat_col,cat_order=cat_order,include_groups=False)
    
    plot = alt.Chart(bar_data).mark_bar() \
        .encode(
            x=alt.X('start:Q', axis=alt.Axis(title=None, format = '%')),
            x2=alt.X2('end:Q'),
            y=alt.Y(f'{question_col}:N', axis=alt.Axis(title=None, offset=5, ticks=False, minExtent=60, domain=False), sort=question_order),
            tooltip=tooltip,
            color=alt.Color(
                f'{cat_col}:N',
                legend=alt.Legend(
                    title='Response',
                    orient='bottom',
                    ),
                scale=color_scale,
            ),
            **({ 'yOffset':alt.YOffset(f'{factor_col}:N', title=None, sort=factor_order),
                 #'stroke': alt.Stroke(f'{factor_col}:N', scale=factor_color_scale, legend=alt.Legend(orient='top')),
                 #'strokeWidth': alt.value(3)
               } if factor_col else {})
        )
    return plot

In [ ]:
e2e_plot({
    'res_col' : 'trust',
    'factor_cols': ['party_preference'],  'filter': {},
    'plot': 'likert_bars',
    'internal_facet': True
},data_file)

In [ ]:
#| export

# Calculate KDE ourselves using a fast libary. This gets around having to do sampling which is unstable
def kde_1d(vc, value_col, ls, scale=False):
    y =  FFTKDE(kernel='gaussian').fit(vc.to_numpy()).evaluate(ls)
    if scale: y*=len(vc)
    return pd.DataFrame({'density': y, value_col: ls})

@stk_plot('density', data_format='raw', continuous=True, factor_columns=3,aspect_ratio=(1.0/1.0),args={'stacked':'bool'})
def density(data, value_col='value',factor_col=None, factor_order=alt.Undefined, factor_color_scale=alt.Undefined, tooltip=[], outer_factors=[], stacked=False, width=800):
    gb_cols = [ c for c in outer_factors+[factor_col] if c is not None ] # There can be other extra cols (like labels) that should be ignored
    
    ls = np.linspace(data[value_col].min()-1e-10,data[value_col].max()+1e-10,200)
    ndata = gb_in_apply(data,gb_cols,cols=[value_col],fn=kde_1d,value_col=value_col,ls=ls,scale=stacked).reset_index()
    
    if stacked:
        
        if factor_col:
            ldict = dict(zip(factor_order, reversed(range(len(factor_order)))))
            ndata.loc[:,'order'] = ndata[factor_col].astype('object').replace(ldict).astype('int')
        
        ndata['density'] /= len(data)
        plot=alt.Chart(ndata).mark_area(interpolate='natural').encode(
                x=alt.X(f"{value_col}:Q"),
                y=alt.Y('density:Q',axis=alt.Axis(title=None, format = '%'),stack='zero'),
                tooltip = tooltip[1:],
                **({'color': alt.Color(f'{factor_col}:N', scale=factor_color_scale, legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(factor_order,width))), 'order': alt.Order('order:O')} if factor_col else {})
            )
    else:
        plot = alt.Chart(
                ndata
            ).mark_line().encode(
                x=alt.X(f"{value_col}:Q"),
                y=alt.Y('density:Q',axis=alt.Axis(title=None, format = '%')),
                tooltip = tooltip[1:],
                **({'color': alt.Color(f'{factor_col}:N', scale=factor_color_scale, legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(factor_order,width)))} if factor_col else {})
            )
    return plot

In [ ]:
e2e_plot({
    'res_col' : 'thermometer',
    'factor_cols': ['party_preference'],  'filter': {},
    'plot': 'density',
    'plot_args': {'stacked': True},
    'internal_facet': True
},data_file,width=500)

In [ ]:
#| export

@stk_plot('violin', data_format='raw', continuous=True, question=True, as_is=True)
def density(data, value_col='value', question_col='question', question_color_scale=alt.Undefined, question_order=alt.Undefined, factor_col=None, factor_order=alt.Undefined, factor_color_scale=alt.Undefined, tooltip=[], outer_factors=[],width=800):
    gb_cols = [ c for c in outer_factors+[question_col]+[factor_col] if c is not None ] # There can be other extra cols (like labels) that should be ignored
    
    ls = np.linspace(data[value_col].min()-1e-10,data[value_col].max()+1e-10,200)
    ndata = gb_in_apply(data,gb_cols,cols=[value_col],fn=kde_1d,value_col=value_col,ls=ls,scale=True).reset_index()
    
    if factor_col:
        ldict = dict(zip(factor_order, reversed(range(len(factor_order)))))
        ndata.loc[:,'order'] = ndata[factor_col].astype('object').replace(ldict).astype('int')

    ndata['density'] /= len(data)
    plot=alt.Chart(ndata).mark_area(interpolate='natural').encode(
            x=alt.X(f"{value_col}:Q"),
            y=alt.Y('density:Q',axis=alt.Axis(title=None, labels=False, values=[0], grid=False),stack='center'),
            row=alt.Row(f'{question_col}:N',header=alt.Header(orient='top',title=None),spacing=5,sort=question_order),
            tooltip = tooltip[1:],
            #color=alt.Color(f'{question_col}:N'),
            **({'color': alt.Color(f'{factor_col}:N', scale=factor_color_scale, legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(factor_order,width))), 'order': alt.Order('order:O')} if factor_col else 
               {'color': alt.Color(f'{question_col}:N', scale=question_color_scale, legend=None)})
        ).properties(width=width,height=70)

    return plot

In [ ]:
e2e_plot({
    'res_col' : 'thermometer',
    'factor_cols': ['party_preference'],  'filter': {},
    'plot': 'violin',
    'internal_facet': True
},data_file, width=650)

In [ ]:
#| export

# Cluster-based reordering
def cluster_based_reorder(X):
    pd = sp.spatial.distance.pdist(X)#,metric='cosine')
    return hierarchy.leaves_list(hierarchy.optimal_leaf_ordering(hierarchy.ward(pd), pd))

@stk_plot('matrix', data_format='longform', requires_factor=True, aspect_ratio=(1/0.8), args={'reorder':'bool'})
def matrix(data, cat_col, value_col='value', cat_order=alt.Undefined, factor_col=None, factor_color_scale=alt.Undefined, factor_order=alt.Undefined, val_format='%', reorder=False, row_normalize=False, tooltip=[]):
    
    fcols = [c for c in data.columns if c not in [value_col,cat_col]]
    if len(fcols)==1 and reorder: # Reordering only works if no external facets
        X = data.pivot(columns=factor_col,index=cat_col).to_numpy()
        cat_order = np.array(cat_order)[cluster_based_reorder(X)]
        factor_order = np.array(factor_order)[cluster_based_reorder(X.T)]
    
    # Find max absolute value to keep color scale symmetric
    dmax = max(-data[value_col].min(),data[value_col].max())
    
    # Draw colored boxes
    base = alt.Chart(data).mark_rect().encode(
            x=alt.X(f'{factor_col}:N', title=None, sort=factor_order),
            y=alt.Y(f'{cat_col}:N', title=None, sort=cat_order),
            color=alt.Color(f'{value_col}:Q', scale=alt.Scale(scheme='redyellowgreen', domainMid=0, domainMin=-dmax, domainMax=dmax),
                legend=alt.Legend(title=None)),
            tooltip=tooltip,
        )
    
    # Add in numerical values
    text = base.mark_text().encode(
        text=alt.Text(f'{value_col}:Q', format=val_format),
        color=alt.condition(
            alt.datum[f'{value_col}:Q']**2 > 1.5,
            alt.value('white'),
            alt.value('black')
        ),
        tooltip=tooltip
    )
    
    return base+text

register_stk_cont_version('matrix')

In [ ]:
e2e_plot({
    'res_col' : 'thermometer',
    'factor_cols': [ 'party_preference'],  'filter': {},
    'plot': 'matrix-cont',
    'internal_facet': True
},data_file)

In [ ]:
e2e_plot({
    'res_col' : 'party_preference',
    'factor_cols': [ 'age_group'],  'filter': {},
    'plot': 'matrix',
    'internal_facet': True
},data_file)

In [ ]:
e2e_plot({
    'res_col' : 'wedge',
    'factor_cols': [ 'party_preference'],  'filter': {},
    'plot': 'matrix-cont',
    'plot_args': { 'reorder':True },
    'internal_facet': True,
    'convert_res': 'continuous',
    'cont_transform': 'center'
},data_file)

In [ ]:
#| export
@stk_plot('lines',data_format='longform', question=False, draws=False, ordered_factor=True, requires_factor=True, args={'smooth':'bool'})
def lines(data, cat_col, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, factor_order=alt.Undefined, smooth=False, width=800, tooltip=[], val_format='.2f',):
    if smooth:
        smoothing = 'basis'
        points = 'transparent'
    else:
        smoothing = 'natural'
        points = True
    plot = alt.Chart(data).mark_line(point=points, interpolate=smoothing).encode(
        alt.X(f'{factor_col}:N', title=None, sort=factor_order),
        alt.Y(f'{value_col}:Q', title=None, axis=alt.Axis(format=val_format)),
        tooltip=tooltip,
        color=alt.Color(f'{cat_col}:N', scale=color_scale, sort=cat_order,
                        legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(cat_order,width)))
    )
    return plot

register_stk_cont_version('lines')

In [ ]:
e2e_plot({
    'res_col' : 'party_preference',
    'factor_cols': ['education'],  'filter': {},
    'plot': 'lines',
    'internal_facet': True,
    'plot_args': { 'smooth':True }
},data_file)

In [ ]:
#| export

def draws_to_hdis(data,vc,hdi_vals):
    gbc = [ c for c in data.columns if c not in [vc,'draw'] ]
    ldfs = []
    for hdiv in hdi_vals:
        ldf_v = data.groupby(gbc,observed=False)[vc].apply(lambda s: pd.Series(list(az.hdi(s.to_numpy(),hdi_prob=hdiv)),index=['lo','hi'])).reset_index()
        ldf_v['hdi']=hdiv
        ldfs.append(ldf_v)
    ldf = pd.concat(ldfs).reset_index(drop=True)
    df = ldf.pivot(index=gbc+['hdi'], columns=ldf.columns[-3],values=vc).reset_index()
    return df

@stk_plot('lines_hdi',data_format='longform', question=False, draws=True, ordered_factor=True, requires_factor=True, args={'hdi1':'float','hdi2':'float'})
def lines_hdi(data, cat_col, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, factor_order=alt.Undefined, width=800, tooltip=[], val_format='.2f', hdi1=0.94, hdi2=0.5):
    
    hdf = draws_to_hdis(data,value_col,[hdi1,hdi2])
    # Draw them in reverse order so the things that are first (i.e. most important) are drawn last (i.e. on top of others)
    # Also draw wider hdi before the narrower
    hdf.sort_values([cat_col,'hdi'],ascending=[False,False],inplace=True)

    plot = alt.Chart(hdf).mark_area(interpolate='basis').encode(
        alt.X(f'{factor_col}:O', title=None, sort=factor_order),
        y=alt.Y('lo:Q',
            axis=alt.Axis(format=val_format),
            title=value_col
            ),
        y2=alt.Y2('hi:Q'),
        color=alt.Color(
            f'{cat_col}:N',
            sort=cat_order,
            scale=color_scale
            ),
        opacity=alt.Opacity('hdi:N',legend=None,scale=to_alt_scale({0.5:0.75,0.94:0.25})),
        tooltip=[
            alt.Tooltip('hdi:N', title='HDI', format='.0%'),
            alt.Tooltip('lo:Q', title='HDI lower', format=val_format),
            alt.Tooltip('hi:Q', title='HDI upper', format=val_format),] + tooltip[1:]
        )
    return plot

register_stk_cont_version('lines_hdi')

In [ ]:
e2e_plot({
    'res_col' : 'party_preference',
    'factor_cols': ['age_group','nationality'],  'filter': {},
    'plot': 'lines_hdi',
    'internal_facet': True,
},data_file)

In [ ]:
#| export
@stk_plot('area_smooth',data_format='longform', question=False, draws=False, ordered=False, ordered_factor=True, requires_factor=True)
def area_smooth(data, cat_col, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, factor_order=alt.Undefined, width=800, tooltip=[]):
    ldict = dict(zip(cat_order, range(len(cat_order))))
    data.loc[:,'order'] = data[cat_col].astype('object').replace(ldict).astype('int')
    #print(data[[cat_col,'order']])
    plot=alt.Chart(data
        ).mark_area(interpolate='natural').encode(
            x=alt.X(f'{factor_col}:O', title=None, sort=factor_order),
            y=alt.Y(f'{value_col}:Q', title=None, stack='normalize',
                 scale=alt.Scale(domain=[0, 1]), axis=alt.Axis(format='%')
                 ),
            order=alt.Order("order:O"),
            color=alt.Color(f"{cat_col}:N",
                legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(cat_order,width)),
                sort=cat_order, scale=color_scale
                ),
            tooltip=tooltip
        )
    return plot

In [ ]:
e2e_plot({
    'res_col' : 'party_preference',
    'factor_cols': ['education','gender'],  'filter': {},
    'plot': 'area_smooth',
    'internal_facet': True
},data_file)

In [ ]:
#| export
def likert_aggregate(x, cat_col, cat_order, value_col):
    
    cc, vc = x[cat_col], x[value_col]
    cats = cat_order
    
    mid, odd = len(cats)//2, len(cats)%2
    
    nonmid_sum = vc[cc !=  cats[mid]].sum() if odd else vc.sum()
    
    #print(len(x),x.columns,x.head())
    pol = ( np.minimum(
                vc[cc.isin(cats[:mid])].sum(),
                vc[cc.isin(cats[mid+odd:])].sum()
            ) / nonmid_sum )

    rad = ( vc[cc.isin([cats[0],cats[-1]])].sum() /
            nonmid_sum )

    rel = 1.0-nonmid_sum/vc.sum()
    
    return pd.Series({ 'polarisation': pol, 'radicalisation':rad, 'relevance':rel})

@stk_plot('likert_rad_pol',data_format='longform', question=False, draws=False, likert=True, requires_factor=True, args={'normalized':'bool'})
def likert_rad_pol(data, cat_col, cat_order=alt.Undefined, value_col='value', factor_col=None, factor_order=alt.Undefined, factor_color_scale=alt.Undefined, normalized=True, width=800, outer_factors=[]):
    #gb_cols = list(set(data.columns)-{ cat_col, value_col }) # Assume all other cols still in data will be used for factoring
    gb_cols = [ c for c in outer_factors+[factor_col] if c is not None ] # There can be other extra cols (like labels) that should be ignored
    
    options_cols = list(data[cat_col].dtype.categories) # Get likert scale names
    likert_indices = data.groupby(gb_cols, group_keys=False, observed=False).apply(likert_aggregate,cat_col=cat_col,cat_order=cat_order,value_col=value_col,include_groups=False).reset_index()
    
    if normalized: likert_indices.loc[:,['polarisation','radicalisation']] = likert_indices[['polarisation','radicalisation']].apply(sps.zscore)
    
    plot = alt.Chart(likert_indices).mark_point().encode(
        x=alt.X('polarisation:Q'),
        y=alt.Y('radicalisation:Q'),
        size=alt.Size('relevance:Q', legend=None, scale=alt.Scale(range=[100, 500])),
        opacity=alt.value(1.0),
        #stroke=alt.value('#777'),
        tooltip=[
            *([alt.Tooltip(f'{factor_col}:N')] if factor_col else []),
            alt.Tooltip('radicalisation:Q', format='.2'),
            alt.Tooltip('polarisation:Q', format='.2'),
            alt.Tooltip('relevance:Q', format='.2')
        ],
        **({'color': alt.Color(f'{factor_col}:N', scale=factor_color_scale, 
                               legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(factor_order,width)))
            } if factor_col else {})
        )
    return plot

In [ ]:
e2e_plot({
  "res_col": "referendum",
  "factor_cols": [
    "electoral_district"
  ],
  "internal_facet": True,
  "plot": "likert_rad_pol",
  "filter": {}
},'../samples/master_bootstrap.parquet',width=600)

In [ ]:
e2e_plot({
    'res_col' : 'trust',
    'factor_cols': ['party_preference'],  'filter': {},
    'plot': 'likert_rad_pol',
    'plot_args': {'normalise':True},
    'internal_facet': True
},data_file)

In [ ]:
#| export
@stk_plot('barbell', data_format='longform', draws=False, requires_factor=True)
def barbell(data, cat_col, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, factor_color_scale=alt.Undefined, factor_order=alt.Undefined, n_datapoints=1, val_format='%', width=800, tooltip=[]):
    
    chart_base = alt.Chart(data).encode(
        alt.X(f'{value_col}:Q', title=None, axis=alt.Axis(format=val_format)),
        alt.Y(f'{cat_col}:N', title=None, sort=cat_order),
        tooltip=tooltip
    )

    chart = chart_base.mark_line(color='lightgrey', size=1, opacity=1.0).encode(
        detail=f'{cat_col}:N'
    )
    selection = alt.selection_point(fields=[factor_col], bind='legend')

    chart += chart_base.mark_point(
        size=50,
        opacity=1,
        filled=True
    ).encode(
        color=alt.Color(f'{factor_col}:N',
            #legend=alt.Legend(orient='right', title=None),
            legend=alt.Legend(orient='top',columns=estimate_legend_columns_horiz(factor_order,width)),
            scale=factor_color_scale,
            sort=factor_order
        ),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    ).add_params(
        selection
    )#.interactive()
    
    return chart
    
register_stk_cont_version('barbell')

In [ ]:
e2e_plot({
    'res_col' : 'trust',
    'factor_cols': ['party_preference'],
    'filter': {},
    'plot': 'barbell-cont',
    'internal_facet': True,
    'convert_res': 'continuous'
},data_file)

In [ ]:
# This does not work because of a Vega lite bug: https://github.com/altair-viz/altair/issues/2369 -> https://github.com/vega/vega-lite/issues/3729
#@stk_plot('geoplot', data_format='longform', continuous=True, requires_factor=True, factor_meta=['topo_feature'])
def geoplot_ideal(data, cat_col, value_col='value', cat_order=alt.Undefined, factor_col=None, val_format='%'):
    
    # TODO: replace with data_format='table' one of these days
    pdf = data.pivot(index=[factor_col], columns=cat_col)
    pdf.columns = pdf.columns.get_level_values(1)
    pdf = pdf.reset_index()
    #print(data)
    
    source = alt.topo_feature(tjson_url, "data")
    plot = alt.Chart(source).transform_lookup(
        lookup = f"properties.{geo_fname}",
        from_ = alt.LookupData(
            data=pdf,
            key=factor_col,
            fields=cat_order
        ),
    ).transform_fold(cat_order,as_=[cat_col,value_col]).mark_geoshape(stroke='white', strokeWidth=0.1).encode(
        tooltip=[alt.Tooltip(f'properties.{ geo_fname}:N', title='maakond'),
                alt.Tooltip(f'{value_col}:Q', title='osakaal', format=val_format)],
        color=alt.Color(
            f'{value_col}:Q',
            scale=alt.Scale(scheme="reds"),
            legend=alt.Legend(format=".0%", title=None, orient='top-left'),
        )
    )
    '''plot = alt.Chart(source).mark_geoshape(stroke='white', strokeWidth=0.1).transform_lookup(
        lookup = f"properties.{geo_fname}",
        from_ = alt.LookupData(
            data=pdf,
            key=factor_col,
            fields=cat_order
        ),
    ).transform_fold(cat_order,as_=[cat_col,value_col]).encode(
        #tooltip=[alt.Tooltip(f'properties.{ geo_fname}:N', title='maakond'),
        #        alt.Tooltip(f'{value_col}:Q', title='osakaal', format='.0%')],
        color=alt.Color(
            f'{value_col}:Q',
            scale=alt.Scale(scheme="reds"),
            legend=alt.Legend(format=".0%", title=None, orient='top-left'),
        ),
        facet=alt.Facet(f'{cat_col}:N')
    )#.facet(f'{cat_col}:N')'''
    

In [ ]:
#| export

@stk_plot('geoplot', data_format='longform', continuous=True, requires_factor=True, factor_meta=['topo_feature'],aspect_ratio=(4.0/3.0))
def geoplot(data, topo_feature, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, val_format='.2f',tooltip=[]):
    
    tjson_url, tjson_meta, tjson_col = topo_feature
    source = alt.topo_feature(tjson_url, tjson_meta)

    plot = alt.Chart(source).mark_geoshape(stroke='white', strokeWidth=0.1).transform_lookup(
        lookup = f"properties.{tjson_col}",
        from_ = alt.LookupData(
            data=data,
            key=factor_col,
            fields=list(data.columns)
        ),
    ).encode(
        tooltip=tooltip, #[alt.Tooltip(f'properties.{tjson_col}:N', title=factor_col),
                #alt.Tooltip(f'{value_col}:Q', title=value_col, format=val_format)],
        color=alt.Color(
            f'{value_col}:Q',
            scale=alt.Scale(scheme="reds"), # To use color scale, consider switching to opacity for value
            legend=alt.Legend(format=val_format, title=None, orient='top-left',gradientThickness=6),
        )
    ).project('mercator')
    return plot

In [ ]:
data_metafile = '../data/master_meta.json'
data_meta = read_json(data_metafile)
    
e2e_plot({
    'res_col' : 'EKRE',
    'factor_cols': ['electoral_district'],
    'filter': {},
    'plot': 'geoplot',
    'internal_facet': True
}, data_file, data_meta=data_meta,width=400)

In [ ]:
#| exporti

# Assuming ns is ordered by unique row values, find the split points
def split_ordered(cvs):
    if len(cvs.shape)==1: cvs = cvs[:,None]
    unique_idxs = np.empty(len(cvs), dtype=np.bool_)
    unique_idxs[:1] = False
    unique_idxs[1:] = np.any(cvs[:-1, :] != cvs[1:, :], axis=-1)
    return np.arange(len(unique_idxs))[unique_idxs]

# Split a series of weights into groups of roughly equal sum
# This algorithm is greedy and does not split values but it is fast and should be good enough for most uses
def split_even_weight(ws, n):
    cws = np.cumsum(ws)
    cws = (cws/(cws[-1]/n)).astype('int')
    return (split_ordered(cws)+1)[:-1]

In [ ]:
# Test
assert (split_even_weight([1,0,1,
                           2,
                           1.5,1,
                           1.5],4) == np.array([3,4,6])).all()

In [ ]:
#| export

def fd_mangle(vc, value_col, factor_col, n_points=10): 
    
    vc = vc.sort_values(value_col)
    
    ws = np.ones(len(vc))
    splits = split_even_weight(ws, n_points)
    
    ccodes, cats = vc[factor_col].factorize()
    
    ofreqs = np.stack([ np.bincount(g, weights=gw, minlength=len(cats))/gw.sum()
                        for g,gw in zip(np.split(ccodes,splits),np.split(ws,splits)) ],axis=0)
    
    df = pd.DataFrame(ofreqs, columns=cats)
    df['percentile'] = np.linspace(0,1,n_points)
    return df.melt(id_vars='percentile',value_vars=cats,var_name=factor_col,value_name='density')

@stk_plot('facet_dist', data_format='raw', continuous=True, factor_columns=3,aspect_ratio=(1.0/1.0),requires_factor=True)
def facet_dist(data, value_col='value',factor_col=None, factor_order=alt.Undefined, factor_color_scale=alt.Undefined, tooltip=[], outer_factors=[]):
    gb_cols = [ c for c in outer_factors if c is not None ] # There can be other extra cols (like labels) that should be ignored
    ndata = gb_in_apply(data,gb_cols,cols=[value_col,factor_col],fn=fd_mangle,value_col=value_col,factor_col=factor_col).reset_index()
    plot=alt.Chart(ndata).mark_area(interpolate='natural').encode(
            x=alt.X(f"percentile:Q",axis=alt.Axis(format='%')),
            y=alt.Y('density:Q',axis=alt.Axis(title=None, format = '%'),stack='normalize'),
            tooltip = tooltip[1:],
            color=alt.Color(f'{factor_col}:N', scale=factor_color_scale, legend=alt.Legend(orient='top')),
            #order=alt.Order('order:O')
    )

    return plot

In [ ]:
e2e_plot({
    'res_col' : 'thermometer',
    'factor_cols': ['party_preference'],  'filter': {},
    'plot': 'facet_dist',
    'internal_facet': True
},data_file)

In [ ]:
#| exporti

# Vectorized multinomial sampling. Should be slightly faster
def vectorized_mn(prob_matrix):
    s = prob_matrix.cumsum(axis=1)
    s = s/s[:,-1][:,None]
    r = np.random.rand(prob_matrix.shape[0])[:,None]
    return (s < r).sum(axis=1)

def linevals(vals, value_col, n_points, dim, cats, ccodes=None, ocols=None, boost_signal=True,gc=False, weights=None):
    ws = weights if weights is not None else np.ones(len(vals))
    
    order = np.lexsort((vals,ccodes)) if dim and gc else np.argsort(vals)
    splits = split_even_weight(ws[order], n_points)
    aer = np.array([ g.mean() for g in np.split(vals[order],splits) ])
    pdf = pd.DataFrame(aer, columns=[value_col])
    
    if dim:
        # Find the frequency of each category in ccodes
        osignal = np.stack([ np.bincount(g, weights=gw, minlength=len(cats))/gw.sum()
                            for g,gw in zip(np.split(ccodes[order],splits),np.split(ws[order],splits)) ],axis=0)
        
        ref_p = osignal.mean(axis=0)+1e-10
        np.random.seed(0) # So they don't change every re-render

        signal = osignal + 1e-10 # So even if values are zero, log and vectorized_mn would work
        if boost_signal: # Boost with K-L, leaving only categories that grew in probability boosted by how much they did
            klv = signal*(np.log(signal/ref_p[None,:]))
            signal = np.maximum(1e-10,klv)
            pdf['kld'] = np.sum(klv,axis=1)

        #pdf[dim] = cats[signal.apply(lambda r: np.random.multinomial(1,r/r.sum()).argmax() if r.sum()>0.0 else 0,axis=1)]
        cat_inds = vectorized_mn(signal)
        pdf[dim] = np.array(cats)[cat_inds]
        pdf['probability'] = osignal[np.arange(len(cat_inds)),cat_inds]

        #pdf[dim] = pdf[cats].idxmax(axis=1)
        #pdf['weight'] = np.minimum(pdf[cats].max(axis=1),pdf['matches'])

    pdf['pos'] = np.arange(0,1,1.0/len(pdf))
    
    if ocols is not None:
        for iv in ocols.index:
            pdf[iv] = ocols[iv]

    return pdf

In [ ]:
#| export

@stk_plot('ordered_population', data_format='raw', continuous=True, factor_columns=3,aspect_ratio=(1.0/1.0),plot_args={'group_categories':'bool'})
def ordered_population(data, value_col='value', factor_col=None, factor_order=alt.Undefined, factor_color_scale=alt.Undefined, tooltip=[], outer_factors=[], group_categories=False):
    
    n_points, maxn = 200, 1000000
    
     # TODO: use weight if available. linevals is ready for it, just needs to be fed in. 
    
    # Sample down to maxn points if exceeding that
    if len(data)>maxn: data = data.sample(maxn,replace=False)
    
    data = data.sort_values(outer_factors)
    vals = data[value_col].to_numpy()

    if factor_col:
        cat_idx, cats = pd.factorize(data[factor_col])
        cats = list(cats)
    else:
        cat_idx, cats = None, []
        
    if outer_factors:
        
        # This is optimized to not use pandas.groupby as it makes it about 2x faster - which is 2+ seconds with big datasets
        
        # Assume data is sorted by outer_factors, split vals into groups by them
        ofids = np.stack([ data[f].cat.codes.values for f in outer_factors ],axis=1)
        splits = split_ordered(ofids)        
        groups = np.split(vals,splits)
        cgroups = np.split(cat_idx,splits) if factor_col else groups
        
        # Perform the equivalent of groupby
        ocols = data.iloc[[0]+list(splits)][outer_factors]
        tdf = pd.concat([linevals(g,value_col=value_col,dim=factor_col, ccodes=gc, cats=cats, n_points=n_points,ocols=ocols.iloc[i,:],gc=group_categories) 
                         for i,(g,gc) in enumerate(zip(groups,cgroups))])

        #tdf = data.groupby(outer_factors,observed=True).apply(linevals,value_col=value_col,dim=factor_col,cats=cats,n_points=n_points,gc=group_categories,include_groups=False).reset_index()
    else:
        tdf = linevals(vals,value_col=value_col,dim=factor_col,ccodes=cat_idx,cats=cats,n_points=n_points, gc=group_categories)
        #tdf = linevals(data,value_col=value_col,cats=cats,dim=factor_col,n_points=n_points,gc=group_categories)
        
    #if boost_signal:
    #    tdf['matches'] = np.minimum(tdf['matches'],tdf['kld']/tdf['kld'].quantile(0.75))

    base = alt.Chart(tdf).encode(
        x=alt.X("pos:Q",
            title="",
            axis=alt.Axis(
                labels=False,
                ticks=False,
                #grid=False
            )
        )
    )
    #selection = alt.selection_multi(fields=[dim], bind='legend')
    line = base.mark_circle(size=10).encode(
        y=alt.Y(f"{value_col}:Q",impute={'value':None}, title='', axis=alt.Axis(grid=True)),
        #opacity=alt.condition(selection, alt.Opacity("matches:Q",scale=None), alt.value(0.1)),
        color=alt.Color(
            f"{factor_col}",
            sort=factor_order,
            scale=factor_color_scale
            ) if factor_col else alt.value('red'),
        tooltip=tooltip+([alt.Tooltip('probability:Q',format='.1%',title='category prob.')] if factor_col else [])
    )#.add_selection(selection)


    rule = alt.Chart().mark_rule(color='red', strokeDash=[2, 3]).encode(
        y=alt.Y('mv:Q')
    ).transform_joinaggregate(
        mv = f'mean({value_col}):Q',
        groupby=outer_factors
    )

    plot = alt.layer(
        rule,
        line,
        data=tdf,
    )
    return plot

In [ ]:
e2e_plot({
    'res_col' : 'thermometer',
    'factor_cols': ['party_preference'],  'filter': {},
    'plot': 'ordered_population',
    #'plot_args': {'group_categories':True},
    'internal_facet': True
},data_file,width=800)

In [ ]:
#| export
@stk_plot('marimekko', data_format='longform', draws=False, group_sizes=True, requires_factor=True, args={'separate':'bool'})
def marimekko(data, cat_col, value_col='value', color_scale=alt.Undefined, cat_order=alt.Undefined, factor_col=None, factor_order=alt.Undefined, factor_color_scale=alt.Undefined, val_format='%', width=800, tooltip=[], outer_factors=[], separate=False):

    #xcol, ycol, ycol_scale = factor_col, cat_col, color_scale
    xcol, ycol, ycol_scale = cat_col, factor_col, factor_color_scale
     
    data['w'] = data['group_size']*data[value_col]
    data.sort_values([xcol,ycol],ascending=[True,False],inplace=True)

    if separate: # Split and center each ycol group so dynamics can be better tracked for all of them
        ndata = data.groupby(outer_factors+[xcol],observed=False)[[ycol,value_col,'w']].apply(lambda df: pd.DataFrame({ ycol: df[ycol], 'yv': df['w']/df['w'].sum(), 'w': df['w']})).reset_index()
        ndata = ndata.merge(ndata.groupby(outer_factors + [ycol],observed=True)['yv'].max().rename('ym').reset_index(),on=outer_factors + [ycol]).fillna({'ym':0.0})
        ndata = ndata.groupby(outer_factors+[xcol],observed=False)[[ycol,'w','yv','ym']].apply(lambda df: pd.DataFrame({ ycol: df[ycol], 'yv': df['yv'], 'w': df['w'].sum(),
                                                                                                                        'y1': (df['ym'].cumsum()- df['ym']/2 - df['yv']/2)/df['ym'].sum(),
                                                                                                                        'y2': (df['ym'].cumsum()- df['ym']/2 + df['yv']/2)/df['ym'].sum(), })).reset_index()
    else: # Regular marimekko
        ndata = data.groupby(outer_factors+[xcol],observed=False)[[ycol,value_col,'w']].apply(lambda df: pd.DataFrame({ ycol: df[ycol], 'w': df['w'].sum(),
                                                                                                                       'yv': df['w']/df['w'].sum(), 
                                                                                                                       'y2': df['w'].cumsum()/df['w'].sum()})).reset_index()
        ndata['y1'] = ndata['y2']-ndata['yv']
    
    ndata = ndata.groupby(outer_factors+[ycol],observed=False)[[xcol,'yv','y1','y2','w']].apply(lambda df: pd.DataFrame({ xcol: df[xcol], 'xv': df['w']/df['w'].sum(), 'x2': df['w'].cumsum()/df['w'].sum(), 'yv':df['yv'], 'y1':df['y1'], 'y2':df['y2']})).reset_index()
    ndata['x1'] = ndata['x2']-ndata['xv']
    

    #selection = alt.selection_point(fields=[yvar], bind="legend")
    STROKE = 0.25
    plot = alt.Chart(ndata).mark_rect(
            strokeWidth=STROKE,
            stroke="white",
            xOffset=STROKE / 2,
            x2Offset=STROKE / 2,
            yOffset=STROKE / 2,
            y2Offset=STROKE / 2,
        ).encode(
            x=alt.X(
                "x1:Q",
                axis=alt.Axis(
                    zindex=1, format="%", title=[f"{xcol} (% of total)", " "], grid=False
                ),
                scale=alt.Scale(domain=[0, 1]),
            ),
            x2="x2:Q",
            y=alt.Y(
                "y1:Q",
                axis=alt.Axis(
                    zindex=1, format="%", title=f"{ycol} (% of total)", grid=False, labels=not separate)
            ),
            y2="y2:Q",
            color=alt.Color(
                f"{ycol}:N",
                legend=alt.Legend(title=None, symbolStrokeWidth=0), #title=f"{yvar}"),
                scale=ycol_scale,
            ),
            tooltip=[
                alt.Tooltip("yv:Q", title=f'{ycol} proportion', format='.1%' ),
                alt.Tooltip("xv:Q", title=f'{xcol} proportion', format='.1%' ),
            ]+tooltip[1:],
            #opacity=alt.condition(selection, alt.value(1), alt.value(0.3)),
        )
        #.add_params(selection)
    
    return plot

In [ ]:
e2e_plot({
    'res_col' : 'trust',
    'factor_cols': ['party_preference'],
    'filter': {},
    'plot_args': {'separate':True},
    'plot': 'marimekko',
    'internal_facet': True,
},data_file)

In [ ]:
# Test convert_res
e2e_plot({
    'res_col' : 'age_group',
    'factor_cols': ['gender'],
    'filter': {},
    'plot': 'boxplots-cont',
    'internal_facet': True,
    'convert_res': 'continuous'
},data_file)

In [ ]:
# Test replace_question and num_values
e2e_plot({
    'res_col' : 'valitsus',
    'factor_cols': ['age_group','party_preference'],
    'filter': {},
    'convert_res': 'continuous',
    'num_values': [0,0,0,1,1],
    'value_name': 'Pr[valitsus==Agree]',
    'value_format': '.0%',
    'plot': 'lines_hdi-cont',
    'internal_facet': True,
    'replace_question': True,
}, data_file, width=800,lazy=False)

In [ ]:
# Test labels
data_file = '../samples/m_bootstrap.parquet'
data_metafile = '../../salk_internal_package/data/master_meta.json'
if data_metafile:
    data_meta = read_json(data_metafile)
e2e_plot({
    'res_col' : 'trust',
    'factor_cols': ['gender','age_group'],
    'filter': {},
    'plot': 'likert_bars',
    'internal_facet': True
}, data_file, width=800, data_meta=data_meta)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()